# 1. 소득 예측 모델

## 모델 만드는 절차
### 전처리 -> 모델 만들기 -> 예측 및 성능 평가

고가 상품 구매 프로모션
- 고가 상품 구매 가능성이 높은 사람을 예측해 프로모션에 드는 시간 및 비용을 절약

저소득층 지원 대상 확대
- 저소득층을 선제적으로 찾아 지원하면 스스로 지원 요건 확인하고 신청하는 경우보다 복지 프로그램 대상을 확대에 용이

In [162]:
import pandas as pd
import numpy as np

df = pd.read_csv('adult.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48842 non-null  int64 
 1   workclass       48842 non-null  object
 2   fnlwgt          48842 non-null  int64 
 3   education       48842 non-null  object
 4   education_num   48842 non-null  int64 
 5   marital_status  48842 non-null  object
 6   occupation      48842 non-null  object
 7   relationship    48842 non-null  object
 8   race            48842 non-null  object
 9   sex             48842 non-null  object
 10  capital_gain    48842 non-null  int64 
 11  capital_loss    48842 non-null  int64 
 12  hours_per_week  48842 non-null  int64 
 13  native_country  48842 non-null  object
 14  income          48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [163]:
# 타겟 변수 전처리
# 연소득이 5만 달러를 초과하는 사람
df['income'].value_counts(normalize = True)

income
<=50K    0.760718
>50K     0.239282
Name: proportion, dtype: float64

In [164]:
# 5만 달러를 초과하면 'high', 그렇지 않으면 'low'로 값을 수정
df['income'] = np.where(df['income'] == '>50K', 'high', 'low')
df['income'].value_counts(normalize = True)

income
low     0.760718
high    0.239282
Name: proportion, dtype: float64

In [165]:
# 불필요한 변수 제거
df = df.drop(columns = 'fnlwgt')

원핫 인코딩 (One-Hot Incoding)
- 변수의 범주가 특정 값이면 1, 그렇지 않으면 0으로 바꾸면 문자 타입 변수를 숫자 타입으로 만드는 것

In [166]:
df_tmp = df[['sex']]
df_tmp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   sex     48842 non-null  object
dtypes: object(1)
memory usage: 381.7+ KB


In [167]:
df_tmp['sex'].value_counts()

sex
Male      32650
Female    16192
Name: count, dtype: int64

In [168]:
# df_tmp의 문자 타입 변수에 원핫 인코딩 적용
df_tmp = pd.get_dummies(df_tmp)
df_tmp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   sex_Female  48842 non-null  bool 
 1   sex_Male    48842 non-null  bool 
dtypes: bool(2)
memory usage: 95.5 KB


In [169]:
df_tmp[['sex_Female', 'sex_Male']].head()
# 원핫코딩 문제 : 0, 1 형태가 아닌, boolean 형태로 나타남

,sex_Female,sex_Male
0,False,True
1,False,True
2,False,True
3,False,True
4,True,False


In [170]:
target = df['income']             # income 추출

df = df.drop(columns = 'income')  # income 제거
df = pd.get_dummies(df)           # 문자 타입 변수 원핫 인코딩

df['income'] = target             # df에 target 삽입
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Columns: 108 entries, age to income
dtypes: bool(102), int64(5), object(1)
memory usage: 7.0+ MB


In [171]:
df.info(max_cols = np.inf)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 108 columns):
 #    Column                                     Non-Null Count  Dtype 
---   ------                                     --------------  ----- 
 0    age                                        48842 non-null  int64 
 1    education_num                              48842 non-null  int64 
 2    capital_gain                               48842 non-null  int64 
 3    capital_loss                               48842 non-null  int64 
 4    hours_per_week                             48842 non-null  int64 
 5    workclass_?                                48842 non-null  bool  
 6    workclass_Federal-gov                      48842 non-null  bool  
 7    workclass_Local-gov                        48842 non-null  bool  
 8    workclass_Never-worked                     48842 non-null  bool  
 9    workclass_Private                          48842 non-null  bool  
 10   workclass_Self-emp-i

## scikit-learn 패키지로 adult 데이터 분할
- test_size : 테스트 세트의 비율. 트레이닝 세트와 테스트 세트의 비율은 보통 7:3 또는 8:2로 정함. 데이터가 많을수록 트레이닝 세트의 비율을 늘리고 반대로 데이터가 적을수록 테스트 세트의 비율을 늘림
- stratify : 범주별 비율을 통일할 변수. stratify에 타겟 변수를 입력하면 트레이닝 세트와 테스트 세트에 타겟 변수의 범주별 비율을 비슷하게 맞춰줌. 타겟 변수를 입력하지 않으면 타겟 변수의 범주별 비율이 데이터 세트마다 달라지므로 평가 결과를 신뢰하기 어려움
- random_state : 난수 초깃값. train_test_split()은 난수를 이용해 데이터를 무작위로 추출하므로 함수를 실행할 때마다 추출되는 데이터가 조금씩 달라짐. 난수를 고정하면 코드를 반복 실행해도 항상 같은 데이터가 추출


In [172]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df,
                                     test_size = 0.3,
                                     stratify = df['income'],
                                     random_state = 1234)

In [173]:
# train
df_train.shape

(34189, 108)

In [174]:
# test
df_test.shape

(14653, 108)

In [175]:
# train
df_train['income'].value_counts(normalize = True)

income
low     0.760713
high    0.239287
Name: proportion, dtype: float64

In [176]:
# test
df_test['income'].value_counts(normalize = True)

income
low     0.760732
high    0.239268
Name: proportion, dtype: float64

## 의사결정나무 모델 생성
sklearn의 tree.DecisionTreeClassifier() 클래스를 이용하면 의사결정나무 모델을 만들 수 있음
- random_state : 난수 초깃값. 변수 선택 과정에서 난수를 이용하기 때문에 코드를 실행할 때마다 결과가 조금씩 달라짐. 난수를 고정하면 코드를 여러 번 실행해도 결과가 항상 같음
- max_depth : 나무의 깊이. 노드를 최대 몇 번까지 분할할지 결정. 숫자가 클수록 노드를 여러 번 분할해 복잡한 모델을 생성. 값을 지정하지 않으면 노드를 최대한 많이 분할

In [177]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(random_state = 1234,  # 난수 고정
                                  max_depth = 3)        # 나무 깊이

## 모델 만들기
df_train에서 예측 변수와 타겟 변수를 각각 추출해 데이터 프레임을 만듦. clf.fit()의 x에는 예측 변수, y에는 타겟 변수를 입력

In [178]:
train_x = df_train.drop(columns = 'income') # 예측 변수 추출
train_y = df_train['income']                # 타겟 변수 추출

model = clf.fit(X = train_x, y = train_y)   # 모델 만들기

## 모델 구조 살펴보기

In [179]:
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.dpi'     : '100',    # 해상도 설정
                     'figure.figsize' : [12, 8]}) # 그래프 크기 설정

tree.plot_tree(model);                            # 그래프 출력

In [180]:
# 가독성
tree.plot_tree(model,
               feature_names = train_x.columns, # 예측 변수명
               class_names = ['high', 'low'],   # 타겟 변수 클래스, 알파벳순
               proportion = True,               # 비율 표기
               filled = True,                   # 색칠
               rounded = True,                  # 둥근 테두리
               impurity = False,                # 불순도 표시
               label = 'root',                  # label 표시 위치
               fontsize = 10);                  # 글자 크기

## 모델을 이용해 예측

In [181]:
test_x = df_test.drop(columns = 'income') # 예측 변수 추출
test_y = df_test['income']                # 타겟 변수 추출

model.predict()를 이용하면 모델을 이용해 새 데이터의 타겟 변수 값을 예측할 수 있음
- model.predict()에 test_x를 입력해 타겟 변수 예측값을 구한 다음 df_test['pred']에 할당
- df_test를 출력하면 가장 오른쪽에 pred가 만들어진 것을 확인 가능
- pred는 모델이 train_x에 들어 있는 예측 변수만 이용해서 구한 값

In [182]:
# 예측값 구하기
df_test['pred'] = model.predict(test_x)
df_test

,age,education_num,capital_gain,capital_loss,hours_per_week,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,...,native_country_Scotland,native_country_South,native_country_Taiwan,native_country_Thailand,native_country_Trinadad&Tobago,native_country_United-States,native_country_Vietnam,native_country_Yugoslavia,income,pred
11712,58,10,0,0,60,False,False,False,False,False,...,False,False,False,False,False,True,False,False,low,low
24768,39,10,0,0,40,False,False,False,False,True,...,False,False,False,False,False,False,False,False,low,low
26758,31,4,0,0,20,False,False,False,False,True,...,False,False,False,False,False,True,False,False,low,low
14295,23,9,0,0,40,False,False,False,False,True,...,False,False,False,False,False,True,False,False,low,low
3683,24,9,0,0,40,False,False,False,False,True,...,False,False,False,False,False,True,False,False,low,low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11985,24,13,0,0,30,False,False,False,False,True,...,False,False,False,False,False,True,False,False,low,low
48445,35,13,10520,0,45,False,False,False,False,True,...,False,False,False,False,False,True,False,False,high,high
19639,41,9,0,0,40,False,False,False,False,True,...,False,False,False,False,False,True,False,False,high,low
21606,29,4,0,0,30,False,False,False,False,True,...,False,False,False,False,False,False,False,False,low,low


## 성능 평가하기
성능 평가 지표는 종류가 다양하고 특징이 서로 달라서 어떤 지표가 높더라도 다른 지표는 낮을 수 있음. 모델을 사용하는 목적에 맞게 평가 기준으로 삼을 지표를 선택해야 함

## confusion matrix
모델이 예측한 값 중 맞은 경우와 틀린 경우의 빈도를 나타낸 컨퓨전 매트릭스 (confusion matrix, 혼동 행렬)를 생성. sklearn.metrics의 confusion_matrix()를 이용하면 컨퓨전 매트릭스를 만들 수 있음. confusion_matrix()에는 다음 파라미터를 입력
- y_true : 타겟 변수
- y_pred : 예측 변수
- labels : 클래스 배치 순서

In [183]:
from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(y_true = df_test['income'], # 실제값
                            y_pred = df_test['pred'],   # 예측값
                            labels = ['high', 'low'])   # 클래스 배치 순서
conf_mat

array([[ 1801,  1705],
       [  582, 10565]], dtype=int64)

sklearn.metrics의 ConfusionMatrixDisplay()를 이용해 컨퓨전 매트릭스로 히트맵을 만들어 격자 위에 표시하고 값이 클수록 셀 색상의 농도를 진하게 표현한 그래프

In [184]:
plt.rcParams.update(plt.rcParamsDefault)                      # 그래프 설정 되돌리기

from sklearn.metrics import ConfusionMatrixDisplay
p = ConfusionMatrixDisplay(confusion_matrix = conf_mat,       # 컨퓨전 매트릭스
                           display_labels = ('high', 'low'))  # 타겟 변수 클래스명

p.plot(cmap = 'Blues')                                        # 컬러맵 적용해 출력
# 컨퓨전 매트릭스 생성 문제

## Accuracy (정확도)
- 모델이 '예측해서 맞춘 비율'을 의미
- 컨퓨전 매트릭스 전체 셀 합계에서 왼쪽 대각선 셀의 합계가 차지하는 비율
- 모델의 성능을 평가할 때 기본적으로 accuracy를 가장 먼저 구함
- 타겟 변수의 클래스별 비율이 불균형하면 신뢰하기 어렵다는 제한점
- 따라서 accuracy의 점수가 높더라도 모델의 성능이 좋아서인지, 자료가 불균형해서인지 판단할 수 없음

In [185]:
# accuracy
import sklearn.metrics as metrics
metrics.accuracy_score(y_true = df_test['income'],  # 실제값
                       y_pred = df_test['pred'])    # 예측값

0.8439227461953184

## Precision (정밀도)
- 모델이 '관심 클래스를 예측해서 맞춘 비율'을 의미
- 컨퓨전 매트릭스 첫 번째 열의 셀 합계에서 위쪽 셀이 차지하는 비율
- ex) 고소득자 예측 모델 : 모델이 income을 high로 예측한 사람 중에서 실제로 high인 사람의 비율
- ex) 당뇨병 예측 모델 : 모델이 발병으로 예측한 사람 중에서 실제 발병한 사람의 비율

In [186]:
# precision
metrics.precision_score(y_true = df_test['income'],   # 실제값
                        y_pred = df_test['pred'],     # 예측값
                        pos_label = 'high')           # 관심 클래스

0.7557700377675199

## Recall (재현율)
- 모델이 '실제 데이터에서 관심 클래스를 찾아낸 비율'을 의미
- 컨퓨전 매트릭스 첫 번째 행의 셀 합계에서 왼쪽 셀이 차지하는 비율
- ex) 고소득자 예측 모델 : income이 실제로 high인 사람 중에서 모델이 high로 예측해서 찾아낸 사람의 비율
- ex) 당뇨병 예측 모델 : 실제 당뇨병 발병자 중에서 모델이 발병으로 예측해서 찾아낸 비율
- recall과 precision은 계산할 때 분모에 놓는 값이 다름. precision은 모델이 '관심 클래스로 예측한 빈도'를 분모에 놓고 구하는 반면, recall은 '실제 관심 클래스의 빈도'를 분모에 놓고 구함

In [187]:
# recall
metrics.recall_score(y_true = df_test['income'],  # 실제값
                     y_pred = df_test['pred'],    # 예측값
                     pos_label = 'high')          # 관심 클래스

0.5136908157444381

## F1 score
- recall과 precision이 모두 중요할 때는 recall과 precision의 크기를 함께 반영한 F1 score를 사용
- recall과 precision의 조화평균으로, 0~1 사이의 값을 가지며 성능이 높을수록 1에 가까운 값이 됨
- recall과 precision을 곱해서 구하기 때문에 둘 중 하나라도 0이면 0이 됨
- accuracy와 달리 타겟 변수의 클래스가 불균형해도 모델의 성능을 잘 표현
- recall과 precision을 고루 반영하고 클래스가 불균형해도 모델의 성능을 잘 나타내므로 여러 모델의 성능을 한 가지 지표로 비교해야 할 때 특히 자주 사용됨

In [188]:
# F1 score
metrics.f1_score(y_true = df_test['income'],  # 실제값
                 y_pred = df_test['pred'],    # 예측값
                 pos_label = 'high')          # 관심 클래스

0.6116488368143997

## 어떤 성능 평가 지표를 사용해야 할까?
- 성능 평가 지표는 특징이 서로 달라서 어떤 지표가 높더라도 다른 지표는 낮을 수 있음. 따라서 모델을 사용하는 목적에 맞게 평가 기준으로 삼을 지표를 선택해야 함
- accuracy는 모델의 일반적인 성능을 나타내므로 항상 살펴봐야 하고, 이데 더해 목적에 따라 precision 또는 recall 중 한 가지 이상을 함께 살펴봐야 함

### precision : 관심 클래스가 분명할 때
- 모델을 사용하는 목적이 타겟 변수의 클래스 중에서 관심을 두는 한쪽 클래스를 정확하게 예측하는 것이라면 precision 기준으로 성능을 평가해야 함
- ex) 고소득자를 예측해 고가의 제품을 홍보하는 경우, 모델이 고소득자로 예측했을 때 얼마나 잘 맞는지 살펴봐야하므로 precision을 기준으로 평가해야 함
- 타겟 변수의 한쪽 클래스에 분명한 관심이 있을 때 precision을 사용

### recall : 관심 클래스를 최대한 많이 찾아내야 할 때
- 모델을 사용하는 목적이 관심 클래스를 최대한 많이 찾아내는 것이라면 recall 기준으로 성능을 평가해야 함
- ex) 전염병에 감염된 사람을 최대한 많이 찾아내 격리해야 한다면, 실제로 전염병에 감염된 사람 중에서 몇 퍼센트를 감염된 것으로 예측하는지 살펴봐야 하므로 recall을 기주능로 평가해야 함

### 관심 클래스로 예측해서 틀릴 때 VS 관심 클래스를 놓칠 때 손실
- 평가 기준으로 삼을 지표를 결정하는 또 다른 방법은 데이터를 '관심 클래스로 예측해서 틀릴 때'의 손실과 '관심 클래스를 놓칠 때'의 손실 중 무엇이 더 큰지를 놓고 판단하는 것
- 데이터를 관심 클래스로 예측해서 틀릴 때 손실이 더 크면 precision, 반대로 관심 클래스를 놓칠 때 손실이 더 크면 recall을 평가 기준으로 사용

### case 1) 고소득자에게 고가 선물을 보내 구매를 독려하는 마케팅
- 관심 클래스로 예측해서 틀릴 때의 손실 : 구매할 가능성이 낮은 저소득자에게 값비싼 선물을 보냄
- 관심 클래스를 놓칠 때의 손실 : 구매할 가능성이 있는 고소득자에게 선물을 보내지 않음
- 이 경우, 데이터를 관심 클래스로 예측해서 틀릴 때의 손실이 더 큼. 따라서 비관심 클래스(negative)를 관심 클래스

## 모델의 성능 지표가 얼마면 될까?